# What this file contains

-   Use the class from 'homework_01.ipynb' to calculate the magnetic field for a given frequency measurement
-   What does $\Omega_{Rabi}$  depend on?

In [1]:
# homework

# what does \Omega Rabi depend on?

# S(1/2)(-1/2) -> D(5/2)(-5/2)

In [2]:
import scipy.constants as c
import numpy as np

hbar = c.hbar # J*s
me = c.electron_mass # kg
qe = c.elementary_charge # C
mu_B = (-qe * hbar) / (2*me) 

class Termsymbol():
    def __init__(self, S, L, J):
        self.hbar = c.hbar # J*s
        self.mu_B = (-qe * hbar) / (2*me) # removed the (-) sign since it messes with the energies corresponding to the MJ values
        self.S = S
        self.L = L
        self.J = J
        self.M_J = np.arange(+self.J, -self.J - 1, -1)
        self.Delta_M_J = self.M_J[1] - self.M_J[0]
        self.G_L = ( (3*self.J*(self.J+1)) + (self.S*(self.S+1)) - (self.L*(self.L+1)) ) / (2*self.J*(self.J+1))
    
    def set_sepcific_MJ(self, MJ):
        self.specific_MJ = MJ
        return self.specific_MJ

    def energy_H_B(self, B): # enter in SI units
        SI = - self.mu_B * self.G_L * self.M_J * B # in SI units (Joule) # a (-) was added to align MJ values with energy values
        self.MHz_per_G = (SI * 1e-6) / (self.hbar*2*np.pi) # calculate to (MHz) 
        # if B entered in [Gauss], then here a transformation would be necessary
    
    def energy_MJ_specific(self):
        index = np.where(self.M_J == self.specific_MJ)[0]  # Find the index where b equals b_value
        if index.size > 0:  # Check if the value exists in b
            self.specific_energy_for_MJ = self.MHz_per_G[index[0]]
            return self.specific_energy_for_MJ   # Return the corresponding value from a
        else:
            return None  # Return None if the value is not found in b

    def get_B(self, Delta_MHZ):
        B_in_Tesla = (-Delta_MHZ * self.hbar * 2 * np.pi) / (self.Delta_M_J * self.mu_B * self.G_L * 1e-6)
        self.B_in_Gauss = B_in_Tesla * 1e4
        return self.B_in_Gauss



In [3]:
# Example of usage:
B = 4.7e-4  # Tesla (4.7 G)

print(f"for B = {B*1e4} Gauss \n")

S1_2 = Termsymbol(0.5, 0, 0.5)  # S1/2
D5_2 = Termsymbol(0.5, 2, 2.5)  # D5/2

S1_2.energy_H_B(B)
D5_2.energy_H_B(B)

print(f"energies S1_2: {S1_2.MHz_per_G}")
print(f"energies D5_2: {D5_2.MHz_per_G} \n")

S1_2.set_sepcific_MJ(-0.5)
S1_2.energy_MJ_specific()

for mj in D5_2.M_J:

    D5_2.set_sepcific_MJ(mj)
    D5_2.energy_MJ_specific()

    print(f"(S1/2(-1/2) -> D5/2({D5_2.specific_MJ})): E_S1_2 at MJ = {S1_2.specific_MJ} = {round(S1_2.specific_energy_for_MJ, 3)} MHz, E_D5_2 at MJ = {D5_2.specific_MJ} = {round(D5_2.specific_energy_for_MJ, 3)} MHz, deltaE = {round(np.absolute(S1_2.specific_energy_for_MJ - D5_2.specific_energy_for_MJ),3)} MHz")

print(f"\nThe effect of the energy splitting is the biggest at S1/2(-1/2) -> D5/2(2.5)")

for B = 4.7 Gauss 

energies S1_2: [ 6.57823512 -6.57823512]
energies D5_2: [ 19.73470536  11.84082322   3.94694107  -3.94694107 -11.84082322
 -19.73470536] 

(S1/2(-1/2) -> D5/2(2.5)): E_S1_2 at MJ = -0.5 = -6.578 MHz, E_D5_2 at MJ = 2.5 = 19.735 MHz, deltaE = 26.313 MHz
(S1/2(-1/2) -> D5/2(1.5)): E_S1_2 at MJ = -0.5 = -6.578 MHz, E_D5_2 at MJ = 1.5 = 11.841 MHz, deltaE = 18.419 MHz
(S1/2(-1/2) -> D5/2(0.5)): E_S1_2 at MJ = -0.5 = -6.578 MHz, E_D5_2 at MJ = 0.5 = 3.947 MHz, deltaE = 10.525 MHz
(S1/2(-1/2) -> D5/2(-0.5)): E_S1_2 at MJ = -0.5 = -6.578 MHz, E_D5_2 at MJ = -0.5 = -3.947 MHz, deltaE = 2.631 MHz
(S1/2(-1/2) -> D5/2(-1.5)): E_S1_2 at MJ = -0.5 = -6.578 MHz, E_D5_2 at MJ = -1.5 = -11.841 MHz, deltaE = 5.263 MHz
(S1/2(-1/2) -> D5/2(-2.5)): E_S1_2 at MJ = -0.5 = -6.578 MHz, E_D5_2 at MJ = -2.5 = -19.735 MHz, deltaE = 13.156 MHz

The effect of the energy splitting is the biggest at S1/2(-1/2) -> D5/2(2.5)


In [4]:
def get_B_from_frequency(frequency, groundstate, MJ_ground, excitedstate, MJ_excited): # frequency in [MHz]
    
    groundstate.set_sepcific_MJ(MJ_ground)
    excitedstate.set_sepcific_MJ(MJ_excited) 

    test_B = 1e-4 # arbitrary magnetic field 

    S1_2_energies = groundstate.energy_H_B(test_B)
    D5_2_energies = excitedstate.energy_H_B(test_B)

    # Get the specific energy for M_J = -0.5 for S1/2 and M_J = -2.5 for D5/2
    E_ground = groundstate.energy_MJ_specific()
    E_excited = excitedstate.energy_MJ_specific()

    # The energy difference if test_B = 1
    deltaE = np.absolute(E_ground - E_excited)

    # We know that the actual difference should be 13.7 MHz, so we solve for B
    scaling_factor = frequency / deltaE  # This is the factor by which we scale the arbitrary B=1 Tesla field

    # print(scaling_factor)

    actual_B = test_B * scaling_factor  # Scale the arbitrary B by this factor

    return actual_B


# test 7 kHz e.g 7000*1e-3 MHz
frequency_to_test = 7 # kHz
frequency_in_MHz = frequency_to_test * 1e-3

B = get_B_from_frequency(frequency_in_MHz, S1_2, -0.5, D5_2, -2.5)
print(f"(Hu et al.) For {frequency_to_test} kHz, the calculated magnetic field (B) is: {round(B * 1e7 ,4)} milli Gauss") # 1e4 would be Gauss

(Hu et al.) For 7 kHz, the calculated magnetic field (B) is: 2.5007 milli Gauss


In [5]:
# fit both data sets
# max: 9.310064484020153 kHz
# min: -2.7222057505828037 kHz
delta_kHz = 12.032270234602956

frequency_our_setup = delta_kHz
frequency_our_setup_in_MHz = frequency_our_setup * 1e-3

B_both_datasets = get_B_from_frequency(frequency_our_setup_in_MHz, S1_2, -0.5, D5_2, -2.5)

print(f"(our setup) For {frequency_our_setup:.2f} kHz, the calculated magnetic field (B) is: {round(B_both_datasets * 1e7 ,4)} milli Gauss") # 1e4 would be Gauss

# fit both dataset 1
# max: 9.310064484020153 kHz
# min: -2.7222057505828037 kHz
delta_kHz = 12.387586782805393

frequency_our_setup = delta_kHz
frequency_our_setup_in_MHz = frequency_our_setup * 1e-3

B_dataset1 = get_B_from_frequency(frequency_our_setup_in_MHz, S1_2, -0.5, D5_2, -2.5)

print(f"(our setup) For {frequency_our_setup:.2f} kHz, the calculated magnetic field (B) is: {round(B_dataset1 * 1e7 ,4)} milli Gauss") # 1e4 would be Gauss

# fit both dataset 2
# max: 8.97694753227419 kHz
# min: -2.743844983462634 kHz
delta_kHz = 11.699153282856994

frequency_our_setup = delta_kHz
frequency_our_setup_in_MHz = frequency_our_setup * 1e-3

B_dataset2 = get_B_from_frequency(frequency_our_setup_in_MHz, S1_2, -0.5, D5_2, -2.5)

print(f"(our setup) For {frequency_our_setup:.2f} kHz, the calculated magnetic field (B) is: {round(B_dataset2 * 1e7 ,4)} milli Gauss") # 1e4 would be Gauss

print()
# fit both dataset 2
# max: 8.97694753227419 kHz
# min: -2.743844983462634 kHz
delta_kHz = 2.8

frequency_our_setup = delta_kHz
frequency_our_setup_in_MHz = frequency_our_setup * 1e-3

B_dataset2 = get_B_from_frequency(frequency_our_setup_in_MHz, S1_2, -0.5, D5_2, -2.5)

print(f"(our setup) For {frequency_our_setup:.2f} kHz, the calculated magnetic field (B) is: {round(B_dataset2 * 1e7 ,4)} milli Gauss") # 1e4 would be Gauss

# print((B_both_datasets - B_dataset1)*1e7)
# print((B_both_datasets - B_dataset2)*1e7)

# fit both dataset 2
# max: 8.97694753227419 kHz
# min: -2.743844983462634 kHz
delta_kHz = 2.9

frequency_our_setup = delta_kHz
frequency_our_setup_in_MHz = frequency_our_setup * 1e-3

B_dataset2 = get_B_from_frequency(frequency_our_setup_in_MHz, S1_2, -0.5, D5_2, -2.5)

print(f"(our setup) For {frequency_our_setup:.2f} kHz, the calculated magnetic field (B) is: {round(B_dataset2 * 1e7 ,4)} milli Gauss") # 1e4 would be Gauss

# fit both dataset 2
# max: 8.97694753227419 kHz
# min: -2.743844983462634 kHz
delta_kHz = 3.1

frequency_our_setup = delta_kHz
frequency_our_setup_in_MHz = frequency_our_setup * 1e-3

B_dataset2 = get_B_from_frequency(frequency_our_setup_in_MHz, S1_2, -0.5, D5_2, -2.5)

print(f"(our setup) For {frequency_our_setup:.2f} kHz, the calculated magnetic field (B) is: {round(B_dataset2 * 1e7 ,4)} milli Gauss") # 1e4 would be Gauss

print() 

# fit both dataset 2
# max: 8.97694753227419 kHz
# min: -2.743844983462634 kHz
delta_kHz = 3.4

frequency_our_setup = delta_kHz
frequency_our_setup_in_MHz = frequency_our_setup * 1e-3

B_dataset2 = get_B_from_frequency(frequency_our_setup_in_MHz, S1_2, -0.5, D5_2, -2.5)

print(f"(our setup) For {frequency_our_setup:.2f} kHz, the calculated magnetic field (B) is: {round(B_dataset2 * 1e7 ,4)} milli Gauss") # 1e4 would be Gauss

# fit both dataset 2
# max: 8.97694753227419 kHz
# min: -2.743844983462634 kHz
delta_kHz = 3.5

frequency_our_setup = delta_kHz
frequency_our_setup_in_MHz = frequency_our_setup * 1e-3

B_dataset2 = get_B_from_frequency(frequency_our_setup_in_MHz, S1_2, -0.5, D5_2, -2.5)

print(f"(our setup) For {frequency_our_setup:.2f} kHz, the calculated magnetic field (B) is: {round(B_dataset2 * 1e7 ,4)} milli Gauss") # 1e4 would be Gauss

# fit both dataset 2
# max: 8.97694753227419 kHz
# min: -2.743844983462634 kHz
delta_kHz = 3.6

frequency_our_setup = delta_kHz
frequency_our_setup_in_MHz = frequency_our_setup * 1e-3

B_dataset2 = get_B_from_frequency(frequency_our_setup_in_MHz, S1_2, -0.5, D5_2, -2.5)

print(f"(our setup) For {frequency_our_setup:.2f} kHz, the calculated magnetic field (B) is: {round(B_dataset2 * 1e7 ,4)} milli Gauss") # 1e4 would be Gauss


print()

# fit both dataset 2
# max: 8.97694753227419 kHz
# min: -2.743844983462634 kHz
delta_kHz = 5.2

frequency_our_setup = delta_kHz
frequency_our_setup_in_MHz = frequency_our_setup * 1e-3

B_dataset2 = get_B_from_frequency(frequency_our_setup_in_MHz, S1_2, -0.5, D5_2, -2.5)

print(f"(our setup) For {frequency_our_setup:.2f} kHz, the calculated magnetic field (B) is: {round(B_dataset2 * 1e7 ,4)} milli Gauss") # 1e4 would be Gauss

# fit both dataset 2
# max: 8.97694753227419 kHz
# min: -2.743844983462634 kHz
delta_kHz = 5.3

frequency_our_setup = delta_kHz
frequency_our_setup_in_MHz = frequency_our_setup * 1e-3

B_dataset2 = get_B_from_frequency(frequency_our_setup_in_MHz, S1_2, -0.5, D5_2, -2.5)

print(f"(our setup) For {frequency_our_setup:.2f} kHz, the calculated magnetic field (B) is: {round(B_dataset2 * 1e7 ,4)} milli Gauss") # 1e4 would be Gauss

# fit both dataset 2
# max: 8.97694753227419 kHz
# min: -2.743844983462634 kHz
delta_kHz = 5.4

frequency_our_setup = delta_kHz
frequency_our_setup_in_MHz = frequency_our_setup * 1e-3

B_dataset2 = get_B_from_frequency(frequency_our_setup_in_MHz, S1_2, -0.5, D5_2, -2.5)

print(f"(our setup) For {frequency_our_setup:.2f} kHz, the calculated magnetic field (B) is: {round(B_dataset2 * 1e7 ,4)} milli Gauss") # 1e4 would be Gauss



# delta_kHz = 2.9, 3.5, 5.3

# 2.9 -> 1.04 +- 0.07
# 3.5 -> 1.25 +- 0.04
# 5.3 -> 1.89 +- +- 0.04


(our setup) For 12.03 kHz, the calculated magnetic field (B) is: 4.2984 milli Gauss
(our setup) For 12.39 kHz, the calculated magnetic field (B) is: 4.4253 milli Gauss
(our setup) For 11.70 kHz, the calculated magnetic field (B) is: 4.1794 milli Gauss

(our setup) For 2.80 kHz, the calculated magnetic field (B) is: 1.0003 milli Gauss
(our setup) For 2.90 kHz, the calculated magnetic field (B) is: 1.036 milli Gauss
(our setup) For 3.10 kHz, the calculated magnetic field (B) is: 1.1074 milli Gauss

(our setup) For 3.40 kHz, the calculated magnetic field (B) is: 1.2146 milli Gauss
(our setup) For 3.50 kHz, the calculated magnetic field (B) is: 1.2503 milli Gauss
(our setup) For 3.60 kHz, the calculated magnetic field (B) is: 1.2861 milli Gauss

(our setup) For 5.20 kHz, the calculated magnetic field (B) is: 1.8576 milli Gauss
(our setup) For 5.30 kHz, the calculated magnetic field (B) is: 1.8934 milli Gauss
(our setup) For 5.40 kHz, the calculated magnetic field (B) is: 1.9291 milli Gauss

# What does $\Omega_{Rabi}$  depend on?

(In the lecture we looked at $\Omega_{effective} = \sqrt{\Omega^2 + \delta^2}$, so the detuning $\delta$ has an effect on  $\Omega_{effective}$)

From Wikipedia

*"The transition dipole moment or transition moment, usually denoted $d_{mm}$ for a transition between an initial state, m, and a final state, n, is the electric dipole moment associated with the transition between the two states. In general the transition dipole moment is a complex vector quantity that includes the phase factors associated with the two states. Its direction gives the polarization of the transition, which determines how the system will interact with an electromagnetic wave of a given polarization, while the square of the magnitude gives the strength of the interaction due to the distribution of charge within the system. The SI unit of the transition dipole moment is the Coulomb-meter (Cm); a more conveniently sized unit is the Debye (D). "*



$$
 \Omega_{0,1} = \frac{\langle 0 | e \mathbf{r} \cdot \mathbf{E}_0 | 1 \rangle}{\hbar}
$$


It depends on:

- matrix dipole moment -> assumption: it also depends on the matrix quadrupole moment
- As can be seen it depends on $\mathbf{E}_0$, e.g the electrical field of the laser. Higher frequency of laser = higher $\Omega_{Rabi}$
- The laser can have a spectrum that includes wavelengths slightly off from the resonant frequency causing detuning ($\Omega_{effective}$).

















